# Deep Neural Network for MNIST Classification

The problem in this notebook  is known as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition.

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [46]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# TensorFLow includes a data provider for MNIST.
# It comes with the tensorflow-datasets module
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data

That's where we load and preprocess our data.

In [40]:
# with_info=True will also provide information about the version, features, number of samples
# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
mnist_dataset, mnist_info = tfds.load(name='mnist', data_dir='D:/Code/Resources/Data/tensorflow_datasets', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)


num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


# scale our data to have inputs between 0 and 1
def scale(image, label):
    image = tf.cast(image, tf.float32) 
    image /= 255.
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model


__Outline the model__

In [41]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


### Choose the optimizer and the loss function

In [42]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [50]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS,  validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.0409 - accuracy: 0.9872 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 11s - loss: 0.0362 - accuracy: 0.9884 - val_loss: 0.0446 - val_accuracy: 0.9860
Epoch 3/5
540/540 - 11s - loss: 0.0327 - accuracy: 0.9899 - val_loss: 0.0507 - val_accuracy: 0.9835
Epoch 4/5
540/540 - 10s - loss: 0.0325 - accuracy: 0.9901 - val_loss: 0.0479 - val_accuracy: 0.9847
Epoch 5/5
540/540 - 11s - loss: 0.0281 - accuracy: 0.9912 - val_loss: 0.0441 - val_accuracy: 0.9858


### Test the model

In [51]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 3s 3s/step - loss: 0.0994 - accuracy: 0.97 - 3s 3s/step - loss: 0.0994 - accuracy: 0.9736

In [76]:
print(f'Test loss: {test_loss:{1}.{2}} Test accuracy: {(100*test_accuracy):{1}.{4}} %')

Test loss: 0.099 Test accuracy: 97.36 %
